In [45]:
# If not running on Google Colab/Drive, skip this
# Run this block first and follow the instructions to authorize mounting
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout  
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.math import confusion_matrix
from math import ceil
import numpy as np
import pandas as pd
import seaborn as sns
import os
import random
import PIL
import matplotlib.pyplot as plt
import pickle

In [0]:
# If not running on Google Colab/Drive, modify this to root, "."
GDRIVE = os.path.join('/', 'content', 'gdrive', 'My Drive')

In [0]:
RANDOM_SEED = 0

OUTPUT_DATA_DIR = os.path.join(GDRIVE, 'data') # we will store train/ valid/ test/ directories here
TRAIN_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'train')
VAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'valid')
TEST_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'test')

WASTE_TYPES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

IMG_HEIGHT = IMG_WIDTH = 256
EPOCHS = 200
BATCH_SIZE = 32

random.seed(RANDOM_SEED)

In [0]:
# So that we can visualize some of the augmented images
TRAIN_AUG_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'train_aug')     

if not os.path.exists(TRAIN_AUG_DATA_DIR):
    os.makedirs(TRAIN_AUG_DATA_DIR)

In [0]:
MODEL_NAME = 'resnet50_augmented'

def create_model():
    base_model = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights=None)
    
    #base_model.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.4),
        Dense(len(WASTE_TYPES))
    ])
    
    model.compile(loss='cosine_similarity', optimizer='adam', metrics=['accuracy'])
    
    return model

In [6]:
model = create_model()
model.load_weights(os.path.join(GDRIVE, f'{MODEL_NAME}.h5'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 12294     
Total params: 23,600,006
Trainable params: 23,546,886
Non-trainable params: 53,120
_________________________________________________________________


In [51]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# using default params: target_size=(256, 256)
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow_from_directory(VAL_DATA_DIR, batch_size=BATCH_SIZE)
# don't shuffle test set, it will be easier to predict then
test_generator = test_datagen.flow_from_directory(TEST_DATA_DIR, batch_size=BATCH_SIZE, shuffle=False)

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

#sample_training_images, _ = next(train_generator)
#plotImages(sample_training_images[:5])

Found 1514 images belonging to 6 classes.
Found 505 images belonging to 6 classes.
Found 508 images belonging to 6 classes.


In [52]:
# Get these values from the above cell, or from prepare_train_test_data.ipynb
total_train = 1514
total_val = 505
total_test = 508
print(total_train, total_val, total_test)

1514 505 508


In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=ceil(total_train / BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=ceil(total_val / BATCH_SIZE),
    initial_epoch=81,
    callbacks=[ModelCheckpoint(os.path.join(GDRIVE, f'{MODEL_NAME}.h5'), save_best_only=True, save_weights_only=True, verbose=1)])

with open(os.path.join(GDRIVE, f'{MODEL_NAME}_history.pickle'), 'wb') as f:
     pickle.dump(history.history, f)

Epoch 82/200
48/48 [==============================] - ETA: 0s - loss: -0.6654 - accuracy: 0.6063 
Epoch 00082: val_loss improved from inf to -0.34051, saving model to /content/gdrive/My Drive/new_resnet50_augmented.h5
48/48 [==============================] - 974s 20s/step - loss: -0.6654 - accuracy: 0.6063 - val_loss: -0.3405 - val_accuracy: 0.2574
Epoch 83/200
48/48 [==============================] - ETA: 0s - loss: -0.6772 - accuracy: 0.6235
Epoch 00083: val_loss did not improve from -0.34051
48/48 [==============================] - 36s 747ms/step - loss: -0.6772 - accuracy: 0.6235 - val_loss: -0.2907 - val_accuracy: 0.2594
Epoch 84/200
48/48 [==============================] - ETA: 0s - loss: -0.6677 - accuracy: 0.6196
Epoch 00084: val_loss did not improve from -0.34051
48/48 [==============================] - 35s 733ms/step - loss: -0.6677 - accuracy: 0.6196 - val_loss: -0.3313 - val_accuracy: 0.2990
Epoch 85/200
48/48 [==============================] - ETA: 0s - loss: -0.6864 - acc

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs_range = range(EPOCHS)

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
# Change title appropriately:
plt.title('Learning Curve, ResNet50 with augmented data')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.show()
plt.savefig(os.path.join(GDRIVE, f'{MODEL_NAME}_learning_curve.png'))
plt.close()

In [54]:
model = create_model()
model.load_weights(os.path.join(GDRIVE, f'{MODEL_NAME}.h5'))
test_predictions = np.argmax(model.predict(test_generator, verbose=1, batch_size=BATCH_SIZE, steps=ceil(total_test / BATCH_SIZE)), axis=-1)

16/16 [==============================] - 3s 173ms/step


In [0]:
test_cm = np.array(confusion_matrix(test_generator.labels, test_predictions))
df_cm = pd.DataFrame(test_cm, WASTE_TYPES, WASTE_TYPES)

plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="YlGnBu")
plt.savefig(os.path.join(GDRIVE, f'{MODEL_NAME}_confusion_matrix.png'))
plt.close()

In [56]:
accuracy = np.trace(test_cm) / np.sum(test_cm)
print(f'Test accruracy: {accuracy * 100:.2f}%')

Test accruracy: 79.33%


In [0]:
# How many augmented images did we produce in the course of training?
!ls /content/gdrive/My\ Drive/data/train_aug | wc -l